In [388]:
import pandas as pd
import re

Load Kenpom

In [389]:
pd.set_option('display.max_columns', 25)
df_html = pd.read_html('https://kenpom.com/index.php?y=2018')

In [390]:
df = df_html[0]
df.columns = df.columns.get_level_values(1)

Get league average Tempo

In [391]:
df_small = df.iloc[:,[1,2,4,5,7,9,11,15,17]].copy()

In [392]:
#Remove line break rows
df_small_fix = df_small.drop(
    [
        df_small.index[40],
        df_small.index[41],
        df_small.index[82],
        df_small.index[83],
        df_small.index[124],
        df_small.index[125],
        df_small.index[166],
        df_small.index[167],
        df_small.index[208],
        df_small.index[209],
        df_small.index[250],
        df_small.index[251],
        df_small.index[292],
        df_small.index[293],
        df_small.index[334],
        df_small.index[335],
    ]
)

In [393]:
df_small_fix['AdjT'] = pd.to_numeric(df_small_fix['AdjT'])
df_small_fix['AdjO'] = pd.to_numeric(df_small_fix['AdjO'])
df_small_fix['AdjD'] = pd.to_numeric(df_small_fix['AdjD'])
df_small_fix['AdjEM'] = pd.to_numeric(df_small_fix['AdjEM'])
df_small_fix['Luck'] = pd.to_numeric(df_small_fix['Luck'])
df_small_fix['OppO'] = pd.to_numeric(df_small_fix['OppO'])
df_small_fix['OppD'] = pd.to_numeric(df_small_fix['OppD'])

In [394]:
laT = df_small_fix['AdjT'].mean()
laT

68.26182336182336

Get league average points per posession

In [421]:
Oppp = df_small_fix['AdjO'].mean()
Dppp = df_small_fix['AdjD'].mean()
Oppp

105.0037037037037

GAMEDAY CALCULATION

In [396]:
team_list = list(df_small_fix['Team'])
new_team_list = []
for team in team_list:
    f = re.split(r'\s+(?=\d)|(?<=\d)\s+', team)[0]
    new_team_list.append(f)

df_small_fix['Team'] = new_team_list

In [475]:
away_team = "Iona"
home_team = "Duke"

df_small_fix[df_small_fix['Team'].isin([home_team, away_team])]

,Team,Conf,AdjEM,AdjO,AdjD,AdjT,Luck,OppO,OppD
2,Duke,ACC,28.86,122.4,93.6,69.7,-0.034,110.4,99.8
142,Iona,MAAC,2.63,110.0,107.3,70.7,-0.008,103.1,105.2


In [476]:
# Get HOME TEAM FIXTURES
hadjO = df_small_fix[df_small_fix['Team'] == home_team]['AdjO'].values[0]
hadjD = df_small_fix[df_small_fix['Team'] == home_team]['AdjD'].values[0]
hadjT = df_small_fix[df_small_fix['Team'] == home_team]['AdjT'].values[0]
hluck = df_small_fix[df_small_fix['Team'] == home_team]['Luck'].values[0]
hOppO = df_small_fix[df_small_fix['Team'] == home_team]['OppO'].values[0]
hOppD = df_small_fix[df_small_fix['Team'] == home_team]['OppD'].values[0]

# Get AWAY TEAM Fixtures
aadjO = df_small_fix[df_small_fix['Team'] == away_team]['AdjO'].values[0]
aadjD = df_small_fix[df_small_fix['Team'] == away_team]['AdjD'].values[0]
aadjT = df_small_fix[df_small_fix['Team'] == away_team]['AdjT'].values[0]
aluck = df_small_fix[df_small_fix['Team'] == away_team]['Luck'].values[0]
aOppO = df_small_fix[df_small_fix['Team'] == away_team]['OppO'].values[0]
aOppD = df_small_fix[df_small_fix['Team'] == away_team]['OppD'].values[0]

In [477]:
#Predicted pace
xp = ( (hadjT * aadjT) / laT )
xp

72.18954544885413

In [478]:
# adjusted effiency differential logic
#if (hadjO > 100):
#    if (aadjD > 100):
#        if (hOppD < 100) or ((aadjD - hOppD) > 0):
#            hadjO = hadjO + (aadjD - hOppD)
            #if (hadjT > 72):
                #hadjO = (hadjO * (hadjT / laT))
        
# adjusted efficiency differential logic
# high powered offenses
if (hadjO > 115):
    #diminishing returns
    if (hadjO - aadjD) > 9.9 and (aadjT < 69):
        if (hadjT < 64.9):
            print("turtling")
            #hadjO = hadjO - 8.31
            hadjO = hadjO + (hOppD - aadjD)
        elif (hadjT > 72):
            print("run n gun")
            hadjO = hadjO - 1.94
        elif (hadjT < 72) and (hadjT > 67):
            if (aadjT < 64) & (hadjO > 125):
                print("Historical offense vs slow team")
                hadjO = hadjO + (aadjD - hOppD)
            else:
                print("blowout, but cap the top off")
                hadjO = hadjO + ((aOppO - aOppD)*(Oppp / laT))
    #breaking point
    elif (hadjO - aadjD) < 9.9 and (aadjT > 70):
        print("run n gun")
        hadjO = (hadjO * hadjT) / laT
        
    if (hadjO - aadjD) > 10 and (aadjT > 70.5):
        if (hadjT > 69):
            if (hadjO - aadjO) < 8:
                print("Should be close-ish")
                hadjO = hadjO - ((hadjO - aadjO)-(hOppD - aadjD))
                
    if (aadjT > 70) and (hadjT > 72):
        if (hadjO - aadjD) > 9.9:
            print("High scoring blowout")
            hadjO = hadjO + ((aOppO - aOppD)*(Oppp / aadjT))
            print(hadjO)
            
# weak offense strong defense teams
elif (hOppD < aadjD) and (aadjT > hadjT):
    if (hadjT < 65) and (hadjO > aadjO):
        hadjO = hadjO
        print("low-scoring control teams")
    elif (hadjO > 112) and (aadjO > hadjO) and (hadjD < 94.5):
        print("upset worthy")
        hadjO = hadjO + (aadjD - hOppD)
    elif (hadjT > 66) and (aadjT < 70):
        print("still needs a nerf")
        hadjO = hadjO + (aOppO - aOppD)

# adjusted home offensive efficiency vs quality of away team offenses
lgadjOh = ((hadjO / aOppO) * Dppp)
#print(lgadjOh)
#xhps = hps * lgadjOa

# away points allowed
apa = aadjD + (aadjD *(.000))

# multiply expected home score by expected away points allowed over league average points per possession
hproj = (lgadjOh * apa) / Oppp
hprojT = round(((hproj * xp) / 100),0)

#print("home team points per possession", hproj, "home team points by predicted tempo", xp, hprojT)
#print(home_team, hprojT)

In [479]:
# adjusted home defensive efficiency differential logic

# Good defensive teams
if (hadjD < 97):
    
    # good defense + fast team
    if (hadjT > 70.5):
        print("Good defense + fast team")
        hadjD = hadjD
    
    # Good defense + slow team
    if (hadjT < 67):
        pass
    
        #slog
        if (hadjD < 91) or (hadjT < 64.9):
            pass
    
    # good defense average pace

    
# home points allowed
hpa = hadjD
    
# adjusted away offensive efficiency differential logic

# Slow as balls and bad offense

# Slow as balls average offense

# slow as balls good offense

# fast and good offense


# Away projected points scored (MORG) adjusted to league average points per possession
lgadjOa = ((aadjO / hOppO) * Dppp)
print(lgadjOa)
#xaps = aps * lgadjOh

# multiply expected home score by expected away points allowed over league average points per possession
aproj = (lgadjOa * hpa) / Oppp
aprojT = round(((aproj * xp) / 100), 0)

#print("away team points per possession", aproj, "away team points by predicted tempo", xp, aprojT)
#print(away_team, aprojT)

104.62495871010361


In [480]:
#home spread
spread = hprojT - aprojT
#projected total
total = hprojT + aprojT

print(home_team, hprojT)
print(away_team, aprojT)
print("Total: ", total)

Duke 92.0
Iona 67.0
Total:  159.0
